In [7]:
from flask import Flask, request, jsonify
from flask_cors import CORS  # Import CORS from flask_cors
# import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
import nltk
import pymysql
import numpy as np

app = Flask(__name__)
CORS(app) 

# # MySQL configuration
# db = pymysql.connect(
#     host='109.106.254.1',
#     user='u690159757_kim',
#     password='*yKa&T6y4#sJ8XvE9j',
#     db='u690159757_kim',
#     cursorclass=pymysql.cursors.DictCursor  
# )
# MySQL configuration
db = pymysql.connect(
    host='mysql5049.site4now.net',
    user='aa0682_movies',
    password='Password1234.',
    db='db_aa0682_movies',
    connect_timeout=8800,
    cursorclass=pymysql.cursors.DictCursor
     
)

In [69]:
db.ping(reconnect=True)
cursor = db.cursor()
cursor.execute(''' SELECT keyword FROM article''')
data = cursor.fetchall()
db.commit()
cursor.close()




In [70]:
data = [list(item.values())[0] for item in data]


In [67]:
uniquwdata= np.unique(data)

In [76]:
data = ",".join(data)
data = data.split()

In [77]:
data

['G,e,n,d,e,r,,',
 ',g,e,n,d,e,r',
 ',e,d,u,c,a,t,i,o,n,,',
 ',g,e,n,d,e,r',
 ',p,e,r,s,p,e,c,t,i,v,e,s,,',
 ',e,x,p,l,o,r,a,t,o,r,y',
 ',s,t,u,d,y,,',
 ',P,h,i,l,i,p,p,i,n,e',
 ',E,d,u,c,a,t,i,o,n',
 ',S,y,s,t,e,m,,',
 ',T,h,e,o,r,i,e,s',
 ',o,f',
 ',L,e,a,r,n,i,n,g,,',
 ',S,D,G,,',
 ',Q,u,e,z,o,n',
 ',C,i,t,y',
 ',U,n,i,v,e,r,s,i,t,y,,,c,a,l,l',
 ',c,e,n,t,e,r',
 ',r,e,p,r,e,s,e,n,t,a,t,i,v,e,s,,',
 ',l,a,b,o,r',
 ',p,r,o,d,u,c,t,i,v,i,t,y,,',
 ',a,n,d',
 ',e,m,p,l,o,y,m,e,n,t,,,s,o,c,i,a,l',
 ',i,s,o,l,a,t,i,o,n,,',
 ',w,o,r,k',
 ',s,a,t,i,s,f,a,c,t,i,o,n,,',
 ',s,t,r,e,s,s,,',
 ',w,o,r,k',
 ',p,r,o,d,u,c,t,i,v,i,t,y,,,e,l,e,c,t,r,o,n,i,c',
 ',w,a,l,l,e,t,,',
 ',a,c,c,u,r,a,c,y',
 ',o,f',
 ',p,a,y,m,e,n,t,,',
 ',o,n,-,t,i,m,e',
 ',d,e,l,i,v,e,r,y,,',
 ',a,c,c,e,s,s,i,b,i,l,i,t,y',
 ',a,n,d',
 ',e,f,f,i,c,i,e,n,c,y,,',
 ',s,e,c,u,r,i,t,y',
 ',a,n,d',
 ',r,e,l,i,a,b,i,l,i,t,y,,,S,a,f,e,t,y',
 ',o,f',
 ',l,i,f,e',
 ',a,t',
 ',s,e,a',
 ',p,r,o,g,r,a,m',
 ',(,S,O,L,A,S,),,',
 ',s,e,a,f,a

In [19]:
db.ping(reconnect=True)
cursor = db.cursor()
cursor.execute('SELECT * FROM movies')
data = cursor.fetchall()

movie_overviews_orig = [row['overview'] for row in data]
movie_id = [row['movie_id'] for row in data]
movie_overviews = [row['overview'] for row in data]
movie_titles = [row['names'] for row in data] 


# Preprocess the movie overviews
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

for n, name in enumerate(movie_overviews):
    temp = name.lower().split(" ")
    temp = [''.join([letter for letter in word if letter.isalnum()]) for word in temp]
    temp = [word for word in temp if word not in stop_words]
    temp = ' '.join(temp)
    movie_overviews[n] = temp

# Calculate cosine similarity
    from sklearn.feature_extraction.text import CountVectorizer

    vectorizer = CountVectorizer().fit_transform(movie_overviews)
    cosine_sim = cosine_similarity(vectorizer)

# Function to get movie recommendations
def get_movie_recommendations( movie_title, similarity_matrix):
    if type(movie_title) == str:
        movie_idx = movie_titles.index(movie_title)
        
    else: 
        movie_idx = movie_title
        print(movie_title)
    similar_movies = list(enumerate(similarity_matrix[movie_idx]))
    similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)
    recommended_movies = []
    
    # Calculate a recommendation score based on similarity (cosine similarity)
    for i in similar_movies:
        if(i[1]< 0.15):
            break
        recommended_movie_title = movie_titles[i[0]]
        movie_description = movie_overviews_orig[i[0]]
        recommended_movies.append({'title': recommended_movie_title, 'description': movie_description,'movie_id': movie_id[i[0]], 'score':i[1]})

    return recommended_movies

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kimbe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
db.ping(reconnect=True) 
with db.cursor() as cursor:
    cursor.execute('SELECT * FROM watchedMovies')
    data = cursor.fetchall()[::-1]
    movie_ids = [row['movie_id'] for row in data]
    movie_ids = np.unique(movie_ids)
    # print(data[0]['movie_id'],"ddata")
    temp=[]
    results=[]
    for i in range(len(movie_ids)):
            recommendations = get_movie_recommendations(i, cosine_sim)[1:]
            if len(recommendations) < 1: continue
            temp.append(recommendations)
            if len(temp) > 5: break
        #  print(data[i]['movie_id'])
    
    # results =[row['movie_id'] for row in temp]
    print('-----',temp,'---------------')
    
    for movie_group in temp:
        for movie in movie_group:
            # Access movie information
            
            movie_id = movie['movie_id']
            results.append(movie_id)
    results = np.unique(results)
    print(results)

0


TypeError: 'int' object is not subscriptable